# Устанавливаем библиотеки

In [1]:
!pip install -q openai
!pip install -q tiktoken
!pip install -q python-docx
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.13.0 requires google-auth-oauthlib<1.1,>=0.5, but you h

In [2]:
import tiktoken
import getpass
import openai
import os
import requests
import ntpath
import re
import io
import json
from googleapiclient.discovery import build
from google.colab import auth
from googleapiclient.http import MediaIoBaseDownload
from docx import Document

In [3]:
# Ключ API пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Обработка текстов инструкций по написанию сопроводительных писем

## Настройка доступа к Google Диску и получение файлов из папки на Google Диске

In [32]:
auth.authenticate_user()                                                        # Авторизации на доступ к Google Диску
drive_service = build('drive', 'v3')                                            # Создание объект service для доступа к Google Drive API

In [33]:
folder_id = '1DeDUAiJmLwWsQHjS0DFcBzfC0oPhPhRH'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
files = results.get('files', [])
files = [file for file in files if file['name'].endswith('.docx')]              # Удаляем файлы не имеющие расширение 'docx'

In [34]:
good_instructions = ['2 сопроводительное письмо.docx',                          # Список инструкций которые мы будем использовать
                     '4 сопроводительное письмо.docx',
                     '7 сопроводительное письмо.docx',
                     '8 сопроводительное письмо.docx']
data = files
files = [item for item in data if item['name'] in good_instructions]

In [35]:
files

[{'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1Dlb2z5HWieqk4F1-DSyR0803Tt3-I8Tl',
  'name': '8 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1Q-YiBo9taElybuByliolzBtaUo-HzwfV',
  'name': '7 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1LLJXhGc4xyrNbVPPddS1dyWlQaFr0kLj',
  'name': '4 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1cmoDNKDCD-bPKrq2P22Sgbytocseaqmf',
  'name': '2 сопроводительное письмо.docx'}]

In [36]:
# Пройдем по списку файлов
for file in files:
    file_id = file['id']
    file_name = file['name']

    # Загрузим файл из Google Диска
    request = drive_service.files().get_media(fileId=file_id)
    file_io = io.BytesIO()
    downloader = MediaIoBaseDownload(file_io, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()

    # Прочитаем текст из файла docx
    document = Document(file_io)
    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text

    file['text'] = text

In [37]:
files[-1]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1cmoDNKDCD-bPKrq2P22Sgbytocseaqmf',
 'name': '2 сопроводительное письмо.docx',
 'text': 'Как джуну написать сопроводительное письмо.\nДжуниор-разработчикам очень важно составлять не только красивое резюме, но и писать классные сопроводительные письма. Рассказываем, что стоит включать в такие письма и как их вообще делать.Обычно сопроводительное письмо необходимо не разработчикам уровня миддл или сеньор, а именно джунам, у которых совсем нет опыта работы, или их уровень явно не дотягивает до интересующей вакансии. Для рекрутеров они — кот в мешке. Их не знают как профессионалов, поэтому совершенно не ясно, что они реально могут предложить работодателю.Возможно, у такого джуна есть ценный опыт в прошлом, или он пришел из смежной профессии и обладает полезными навыками. Или может быть он просто классный и очень ответственный разработчик, хоть и без большого количества опы

## Считаем количество токенов в каждом ключе 'text' списка 'files'

In [ ]:
# Создаем экземпляр токенизатора
encoding = tiktoken.get_encoding("cl100k_base")

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['text']))                            # Токенизируем текст и считаем количество токенов
    file['token'] = token_count                                                 # Записываем количество токенов в ключ 'token'

    print("name:", file['name'])
    print('token:', file['token'],'\n' )

name: 8 сопроводительное письмо.docx
token: 3094 

name: 7 сопроводительное письмо.docx
token: 2091 

name: 4 сопроводительное письмо.docx
token: 1633 

name: 2 сопроводительное письмо.docx
token: 3072 



In [ ]:
files[-1]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1cmoDNKDCD-bPKrq2P22Sgbytocseaqmf',
 'name': '2 сопроводительное письмо.docx',
 'text': 'Как джуну написать сопроводительное письмо.\nДжуниор-разработчикам очень важно составлять не только красивое резюме, но и писать классные сопроводительные письма. Рассказываем, что стоит включать в такие письма и как их вообще делать.Обычно сопроводительное письмо необходимо не разработчикам уровня миддл или сеньор, а именно джунам, у которых совсем нет опыта работы, или их уровень явно не дотягивает до интересующей вакансии. Для рекрутеров они — кот в мешке. Их не знают как профессионалов, поэтому совершенно не ясно, что они реально могут предложить работодателю.Возможно, у такого джуна есть ценный опыт в прошлом, или он пришел из смежной профессии и обладает полезными навыками. Или может быть он просто классный и очень ответственный разработчик, хоть и без большого количества опы

## Антотации из инструкций по написанию резюме



In [ ]:
system_cont = '''
Ты - эксперт и HR-специалист. Ты отлично структуририруешь текст и выделяешь ключевые моменты.
Возьми на себя роль редактора, которому предоставили ТЕКСТ ИНСТРУКЦИИ для получения анотации, и сконцентрируйся только на его содержании, игнорируя вопросы форматирования и дизайна.
В своей анотации выдели из полученного ТЕКСТ ИНСТРУКЦИИ основные идеи и требования.
Оставь только самые важные пункты по содержанию сопроводительные письма.
Ты должен писать в единственном числе повелительного наклонения.
Размер анотации должен быть не меньше 700 символов.
'''

Так, как у части инструкций текст больше 4к токенов, то выбирам модель  'gpt-3.5-turbo-16k'

In [ ]:
# Проходим по каждому элементу списка
for file in files:
    messages = [{"role": "system", "content": system_cont},
                {"role": "user", "content": 'ТЕКСТ ИНСТРУКЦИИ: ' + file['text']}]

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-16k",
                  messages=messages,
                  temperature=0.2
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:', file['annotation'],'\n' )

name: 8 сопроводительное письмо.docx
annotation: Основные идеи и требования:

- Сопроводительное письмо - это дополнение к резюме, которое помогает привлечь внимание работодателя и обойти алгоритмы джоб-сайтов.
- Объем сопроводительного письма должен быть от 500 до 1000 знаков.
- В письме нужно выразить интерес к вакансии и компании, рассказать о своих навыках и опыте работы, упомянуть достижения на предыдущем месте работы.
- Заключение письма должно содержать благодарность и приглашение на интервью.
- Контакты нужно указать в конце письма для подтверждения их актуальности.
- Сопроводительное письмо должно быть написано в официально-деловом стиле.
- Важно показать себя экспертом и убедить работодателя в том, что вы принесете пользу компании.
- Структура письма может варьироваться, но обычно включает приветствие, интерес к вакансии, описание навыков, упоминание достижений, заключение и контакты.
- Письмо должно быть кратким и информативным.
- Необходимо избегать нестандартных приветстви

In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1Dlb2z5HWieqk4F1-DSyR0803Tt3-I8Tl',
 'name': '8 сопроводительное письмо.docx',
 'text': 'Как правильно составить сопроводительное письмо программистуДавайте разберемся, зачем нужно сопроводительное письмо и как его написать так, чтобы вас захотели взять на работу.Несмотря на то что сопроводительное письмо или cover letter составляется в свободной форме, мало кто действительно умеет писать его правильно и убедительно. Особенно это относится к джунам и чуть меньше — к мидлам.Что такое сопроводительное письмоСопроводительное письмо — это небольшой текст, который идет дополнением к резюме. В нем вы должны в двух словах объяснить эйчару, зачем и почему вас брать на работу.Отличительные особенности:Объем: 500 — 1000 знаковСтиль: официально-деловой*О чем: какую пользу вы принесете компании*чаще всего, но не всегдаНемного статистикиСогласно  ResumeLab, около 83% работодателей 

### Считаем количество токенов в анотациях

In [ ]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 8 сопроводительное письмо.docx
token: 555 

name: 7 сопроводительное письмо.docx
token: 471 

name: 4 сопроводительное письмо.docx
token: 250 

name: 2 сопроводительное письмо.docx
token: 384 

total_tokens = 1660


In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1Dlb2z5HWieqk4F1-DSyR0803Tt3-I8Tl',
 'name': '8 сопроводительное письмо.docx',
 'text': 'Как правильно составить сопроводительное письмо программистуДавайте разберемся, зачем нужно сопроводительное письмо и как его написать так, чтобы вас захотели взять на работу.Несмотря на то что сопроводительное письмо или cover letter составляется в свободной форме, мало кто действительно умеет писать его правильно и убедительно. Особенно это относится к джунам и чуть меньше — к мидлам.Что такое сопроводительное письмоСопроводительное письмо — это небольшой текст, который идет дополнением к резюме. В нем вы должны в двух словах объяснить эйчару, зачем и почему вас брать на работу.Отличительные особенности:Объем: 500 — 1000 знаковСтиль: официально-деловой*О чем: какую пользу вы принесете компании*чаще всего, но не всегдаНемного статистикиСогласно  ResumeLab, около 83% работодателей 

## Попытка №2

In [ ]:
files_2 = files.copy()

In [ ]:
system_cont_2 = '''
Ты профессиональный HR-менеджер и хорошо структурируешь тексты, умеешь выделять самые важные тезисы.
Сконцентрируйся только на содержании ТЕКСТ ИНСТРУКЦИИ, игнорируя вопросы форматирования и дизайна.
Сократи ТЕКСТ ИНСТРУКЦИИ до 500 слов, оставь по одному абзацу на каждый тезис.
Пиши в единственном числе повелительного наклонения.
'''

In [ ]:
# Проходим по каждому элементу списка
for file in files_2:
    messages = [{"role": "system", "content": system_cont_2},
                {"role": "user", "content": 'ТЕКСТ ИНСТРУКЦИИ: ' + file['text']}]

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo",
                  messages=messages,
                  temperature=0.2
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:\n', file['annotation'],'\n' )

name: 8 сопроводительное письмо.docx
annotation:
 - Сопроводительное письмо — небольшой текст, дополнение к резюме, в котором нужно объяснить, зачем и почему вас брать на работу.
- Согласно исследованиям, хорошее сопроводительное письмо может повлиять на решение работодателя и увеличить шансы на приглашение на собеседование.
- Сопроводительное письмо помогает обойти алгоритмы джоб-сайтов и заинтересовать работодателя.
- Важно правильно структурировать сопроводительное письмо, включая приветствие, интерес к вакансии, описание своих навыков и достижений, заключение и контакты.
- Пишите сопроводительное письмо так, как разговариваете в обычной жизни, но с долей деловой этики.
- Покажите себя экспертом и расскажите о пользе, которую принесете компании.
- Используйте лайфхак и попросите ChatGPT составить подробное сопроводительное письмо, если затрудняетесь с описанием своих навыков и достижений. 

name: 7 сопроводительное письмо.docx
annotation:
 - В письме укажите о будущих применениях св

In [ ]:
files_2[-1]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1cmoDNKDCD-bPKrq2P22Sgbytocseaqmf',
 'name': '2 сопроводительное письмо.docx',
 'text': 'Как джуну написать сопроводительное письмо.\nДжуниор-разработчикам очень важно составлять не только красивое резюме, но и писать классные сопроводительные письма. Рассказываем, что стоит включать в такие письма и как их вообще делать.Обычно сопроводительное письмо необходимо не разработчикам уровня миддл или сеньор, а именно джунам, у которых совсем нет опыта работы, или их уровень явно не дотягивает до интересующей вакансии. Для рекрутеров они — кот в мешке. Их не знают как профессионалов, поэтому совершенно не ясно, что они реально могут предложить работодателю.Возможно, у такого джуна есть ценный опыт в прошлом, или он пришел из смежной профессии и обладает полезными навыками. Или может быть он просто классный и очень ответственный разработчик, хоть и без большого количества опы

### Считаем количество токенов в анотациях

In [ ]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files_2:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 8 сопроводительное письмо.docx
token: 380 

name: 7 сопроводительное письмо.docx
token: 589 

name: 4 сопроводительное письмо.docx
token: 303 

name: 2 сопроводительное письмо.docx
token: 379 

total_tokens = 1651


# Сумаризациия анотаций

## Объединяем все анотации одного примера в единый текст.

In [ ]:
annotations = ''

for file in files:
    annotations = annotations + ' ' + file['annotation']

In [ ]:
print(annotations)

 - Сопроводительное письмо — небольшой текст, дополнение к резюме, в котором нужно объяснить, зачем и почему вас брать на работу.
- Согласно исследованиям, хорошее сопроводительное письмо может повлиять на решение работодателя и увеличить шансы на приглашение на собеседование.
- Сопроводительное письмо помогает обойти алгоритмы джоб-сайтов и заинтересовать работодателя.
- Важно правильно структурировать сопроводительное письмо, включая приветствие, интерес к вакансии, описание своих навыков и достижений, заключение и контакты.
- Пишите сопроводительное письмо так, как разговариваете в обычной жизни, но с долей деловой этики.
- Покажите себя экспертом и расскажите о пользе, которую принесете компании.
- Используйте лайфхак и попросите ChatGPT составить подробное сопроводительное письмо, если затрудняетесь с описанием своих навыков и достижений. - В письме укажите о будущих применениях своих знаний в компании, где желаете работать.
- Сформулируйте структуру письма, чтобы контекст был кра

## Ручная правка списка тезисов
Промтом удалить "мусорные" тезисы не получается, сделаем это ручками, и вернем обратно

In [ ]:
annotations = '''
Объем сопроводительного письма должен быть от 180 до 220 слов.
Стиль письма должен быть официально-деловым.
В письме нужно подчеркнуть свой интерес к вакансии и компании.
Важно кратко описать свои навыки и опыт работы.
Рассказать о своих достижениях на предыдущем месте работы и о том, какую пользу принесли работодателю.
Заключение письма должно содержать благодарность за уделенное время и готовность к дальнейшему общению.
Контактные данные нужно указать в письме, чтобы подтвердить их актуальность.
Структура письма может варьироваться, но базово она состоит из: приветствия, интереса к вакансии, описания навыков, рассказа о достижениях, заключения и контактов.
Не стоит преуменьшать свои достижения и навыки.
Письмо должно продавать вас как специалиста и заинтересовать работодателя.
Содержание сопроводительного письма должно быть кратким и лаконичным.
В начале письма необходимо приветствовать рекрутера и представиться, указав желаемую должность.
Объяснить причину написания письма в данную компанию и почему нанимателю будет выгодно нанять именно вас.
Рассказать о профессиональных умениях и опыте работы, упомянув проекты, в которых вы участвовали.
Указать контактные данные для связи и призывать к действию, например, позвонить и назначить собеседование.
Избегать фраз-клише и использовать конкретные примеры, проекты и выполненные задачи.
Проверять письмо на грамотность и стилистическую правильность.
Создавать аккуратное и структурированное содержание.
Указывать языки разработки и детали работы с технологиями.
Пиши в активном залоге, описывая выполненные задачи и достижения.
Упоминать личностные качества и навыки работы в команде. Основные идеи и требования из текста инструкции для написания эффективного сопроводительного письма для работы в IT-сфере:
Изучите требования работодателя перед написанием письма.
Составьте ясный и лаконичный заголовок, отражающий суть письма.
Во введении кратко представьте себя и свои навыки, а также объясните, почему хотите работать именно в этой компании.
Подчеркните свой опыт и достижения, особенно те, которые релевантны для данной позиции.
Покажите свою мотивацию и энтузиазм, объяснив, почему хотите работать именно в этой компании.
'''

## Сумаризируем сумму анотаций

### Вариант 1, не забудем ничего
ничего не выбрасываем

In [ ]:
system_cont_3 = '''
Ты профессиональный копирайтер.
Необходимо объединить похожие по смыслу тезисы.
При необходимости перефразируй  тезисы.
Удали все пункты по оформлению, форматированию и целям письма.
Оставь тезисы как писать письмо, а не для чего оно нужно.
Надо откорректировать список, самые значимые тезисы поднять выше.
'''

In [ ]:
messages = [{"role": "system", "content": system_cont_3},
            {"role": "user", "content": annotations}]
completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=messages,
              temperature=0.1
              )
summ_annotation = completion['choices'][0]['message']['content']

In [ ]:
print(summ_annotation)

- Сопроводительное письмо является важным дополнением к резюме и позволяет объяснить, почему вас нужно выбрать на работу.
- Хорошо составленное сопроводительное письмо может повлиять на решение работодателя и увеличить шансы на приглашение на собеседование.
- Сопроводительное письмо помогает привлечь внимание работодателя и обойти алгоритмы джоб-сайтов.
- Важно структурировать письмо, включая приветствие, интерес к вакансии, описание навыков и достижений, заключение и контакты.
- Пишите письмо в разговорном стиле с долей деловой этики.
- Покажите себя экспертом и объясните, какую пользу принесете компании.
- Укажите о будущих применениях своих знаний в компании, где хотите работать.
- Используйте примеры, проекты и выполненные задачи, чтобы привлечь внимание работодателя.
- Избегайте фраз-клише и описывайте свои умения и навыки работы в команде.
- Проверьте письмо на грамотность и стилистическую правильность.
- Создайте аккуратное и структурированное содержание, выбирайте информативный

### Вариант 2, краткость сестра таланта
оставляем только 12 самых важных пунктов

In [ ]:
system_cont_4 = '''
Ты профессиональный копирайтер.
Объедини похожие по смыслу тезисы. При необходимости перефразируй тезисы.
Откорректируй список, самые значимые тезисы подими выше.
Удали все пункты по оформлению, форматированию и целям письма.
Оставь тезисы как писать письмо, а не для чего оно нужно.
Оставь только 12 самых важных пунктов, относящихся к содержанию.
Пиши в единственном числе повелительного наклонения.
'''

In [ ]:
messages = [{"role": "system", "content": system_cont_4},
            {"role": "user", "content": annotations}]
completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=messages,
              temperature=0
              )
summ_annotation = completion['choices'][0]['message']['content']

In [ ]:
print(summ_annotation)

- Напишите сопроводительное письмо, чтобы объяснить, зачем и почему вас нужно взять на работу.
- Структурируйте письмо, включая приветствие, интерес к вакансии, описание навыков и достижений, заключение и контакты.
- Покажите себя экспертом и расскажите о пользе, которую принесете компании.
- Укажите о будущих применениях своих знаний в компании, где желаете работать.
- Расскажите о своих профессиональных умениях и опыте работы, упомяните проекты, в которых вы участвовали.
- Укажите контактные данные для связи и призывайте действовать, позвонить вам и назначить собеседование.
- Будьте краткими и конкретными, используйте примеры, проекты и выполненные задачи, чтобы привлечь внимание рекрутера.
- Избегайте фраз-клише, опишите свои умения и навыки работы в команде, с программами и методы решения сложных ситуаций.
- Проверьте письмо на грамотность и стилистическую правильность, избегайте пунктуационных ошибок.
- Создавайте аккуратное и структурированное содержание, выбирайте емкий и информ

# Написание сопроводительного письма по тексту резюме и вакансии

In [4]:
auth.authenticate_user()                                                        # Авторизации на доступ к Google Диску
drive_service = build('drive', 'v3')                                            # Создание объект service для доступа к Google Drive API

Текст резюме берем из текстового документа `Резюме общая база` в рабочей папке `HR-документы`

In [5]:
file_url = 'https://docs.google.com/document/d/1d5rancSjPnGdScZ3wjjbLak-xZPkvcY8NRm5-Wvw9-8/export?format=txt'
response = requests.get(file_url)
resume = response.text

Список с сылками на файлы с текстами вакансий полученных с `HH.ru` и `telegram`-каналов

In [6]:
folder_id = '1h5v3HDBX-A9KaL2QMbpbgdY9rOB2B2rD'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
vacancies = results.get('files', [])
vacancies = [vacanc for vacanc in vacancies if vacanc['name'].endswith('.txt')] # Удаляем файлы не имеющие расширение 'txt'

In [7]:
vacancies

[{'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1fHhz9mwUk80msizLUH87A64knKEXWHbG',
  'name': 'Вакансия01.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1YaHhrWfkU5e-zZTCSYUHGecbPngP15tM',
  'name': 'Вакансия05.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1D6xq4BLafRWcGlejTtshGeN8b8eOubsy',
  'name': 'Вакансия06.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1pq7gHI8oxc5V1mNkoecZ_i6C1Nnq1Otl',
  'name': 'Вакансия03.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '14O5mOapCGyhoTd31eb1WJRrvHWYzO2lc',
  'name': 'Вакансия04.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1RC0jRoRMKdEtm43Ix_65k2x2UWC70r-z',
  'name': 'Вакансия07.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '10saGxyc6q_LHlbB3qOSqsqbi69EBkBew',
  'name': 'Вакансия02.txt'}]

Сохраняем тексты из вакансий в список `vacancies`

In [8]:
# Пройдем по списку файлов
for vacancy in vacancies:
    vacancy_id = vacancy['id']
    vacancy_name = vacancy['name']

    # Загрузим файл из Google Диска
    request = drive_service.files().get_media(fileId=vacancy_id)
    vacancy_io = io.BytesIO()
    downloader = MediaIoBaseDownload(vacancy_io, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()

    # Прочитаем текст из файла
    vacancy_io.seek(0)  # Перемотаем указатель в начало файла
    text = vacancy_io.read().decode('utf-8').strip()

    vacancy['text'] = text

In [9]:
vacancies[0]

{'kind': 'drive#file',
 'mimeType': 'text/plain',
 'id': '1fHhz9mwUk80msizLUH87A64knKEXWHbG',
 'name': 'Вакансия01.txt',
 'text': 'HR-холдинг Ventra\r\n\r\nО компании\r\nVentra — HR-холдинг, предлагающий инновационные решения в области аутсорсинга бизнес-процессов, HR-консалтинга и гибкой занятости. Мы обладаем широкими возможностями для решения ваших задач: полный спектр HR-сервисов, услуги аутсорсинга продаж, ИТ, процессов складской логистики и производственных функций. В июне 2022 холдинг приобрел бизнес компании Kelly в России, объединив сильнейшие на российском рынке команды в области решений по управлению персоналом и аутсорсинга. \r\n20\r\nлет компания на рынке\r\n10000\r\nсотрудников на проектах компании\r\n350\r\nклиентов из ТОП-10 индустрий\r\n5+\r\nлет каждый клиент сотрудничает с Ventra\r\n70%\r\nклиентов пользуются 2 и более услугами\r\n450 тыс.\r\nтранзакций в сутки в платформе Ventra Portal\r\n300\r\nтоп-менеджеров в закрытом клубе Ventra Lab\r\n Направления\r\nМы учитыв

In [10]:
system_promt = '''
Ты HR-специалист и должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.
В сопроводительном письме обязательно обрати внимание на ключевые навыки, языки программирования, библиотеки и методы исследования, совпадающие в ВАКАНСИИ и в РЕЗЮМЕ.
Подчеркни насколько опыт соискателя в РЕЗЮМЕ подходит требованиям ВАКАНСИИ.
Напиши текст не меньше 180 и не больше 220 слов.
При написании письма придерживайся следующих инструкций:

'''

## Вариант 1

Текст `system_promt_1` написан с использованием антотции полученной из `7 сопроводительное письмо.docx`

In [20]:
system_promt_1 = '''
Ты HR-специалист и должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.
При написании письма придерживайся следующих инструкций:
- В письме укажите о будущих применениях своих знаний в компании, где желаете работать.
- Сопроводительное письмо должно быть кратким и лаконичным, сформулировано с нового абзаца с отступом.
- В письме нужно объяснить причину, почему вы пишете именно в эту компанию и почему работодателю будет выгодно нанять вас.
- Укажите свои профессиональные умения и опыт работы, упомяните проекты, в которых вы участвовали.
- Укажите контактные данные для связи и призывайте к действию - позвонить вам и назначить собеседование.
- Используйте речевые обороты, прямое обращение по имени, упоминайте, откуда вы узнали о вакансии и делайте комплименты фирме.
- Проверьте письмо на грамотность и пунктуационные ошибки.
- Оформите сопроводительное письмо в печатном виде, предпочтительно в PDF-формате.
- Создайте аккуратное и структурированное содержание, используйте емкий и информативный стиль написания.
- Для программиста: укажите языки разработки, уделите внимание деталям, добавьте ссылки на разработанные приложения и список технологий, которые вы применяли.
- Пишите в активном залоге, описывайте выполненные вами задачи и укажите личностные качества.
'''

In [21]:
# messages = [{"role": "system", "content": system_promt_1},
#             {"role": "user", "content": 'ТЕКСТ РЕЗЮМЕ: ' + resume + 'ТЕКСТ ВАКАНСИИ: ' + vacancies[0]['text']}]
# completion = openai.ChatCompletion.create(
#               model="gpt-3.5-turbo",
#               messages=messages,
#               temperature=0
#               )
# cover_letter_1 = completion['choices'][0]['message']['content']
# print(cover_letter_1)

In [22]:
# Проходим по каждому элементу списка
for vacancy in vacancies:
    messages = [{"role": "system", "content": system_promt_1},
                {"role": "user", "content": 'ТЕКСТ РЕЗЮМЕ: ' + resume + 'ТЕКСТ ВАКАНСИИ: ' + vacancy['text']}]

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo",
                  messages=messages,
                  temperature=0
                  )
    vacancy['cover_letter_1'] = completion['choices'][0]['message']['content']

    print("name:", vacancy['name'])
    print('cover_letter_1:', vacancy['cover_letter_1'],'\n' )

name: Вакансия01.txt
cover_letter_1: Уважаемый HR-холдинг Ventra,

Я обращаюсь к вам с большим интересом к вакансии Data Scientist. Я Максим Иванов, мотивированный Junior DS-инженер с годовым опытом в анализе данных и машинном обучении. Я хотел бы применить свои знания и навыки в вашей компании, где я смогу развиваться и углублять свои навыки в области Data Science.

В моем текущем опыте работы в компании XYZ Corp, я занимался проектированием и реализацией моделей предсказательного анализа, обработкой и анализом больших данных с использованием Python, R, SQL и Tableau. Я также строил и отлаживал алгоритмы машинного обучения и AI. Мой опыт позволил мне развить навыки работы с Python, R, SQL, а также использование библиотек Pandas, NumPy и Scikit-learn.

Я обладаю отличными навыками анализа данных и проблемного решения, вниманием к деталям и способностью работать в мультидисциплинарных проектах. Я быстро обучаюсь и адаптируюсь к новым технологиям. Кроме того, я обладаю хорошими коммуника

## Вариант 2

Текст `system_promt_2` написан с использованием антотции полученной из `2 сопроводительное письмо.docx`

In [23]:
system_promt_2 = '''
Ты HR-специалист и должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.
В сопроводительном письме обязательно обрати внимание на ключевые навыки, языки программирования, библиотеки и методы исследования, совпадающие в ВАКАНСИИ и в РЕЗЮМЕ.
Подчеркни насколько опыт соискателя в РЕЗЮМЕ подходит требованиям ВАКАНСИИ.
Напиши текст не меньше 180 и не больше 220 слов.
При написании письма придерживайся следующих инструкций:
- Сопроводительное письмо важно для джунов, чтобы проявить себя и привлечь внимание рекрутеров.
- В письме нужно рассказать о мотивации, профессиональных навыках и преимуществах кандидата.
- Подгоняйте письмо под каждую конкретную вакансию.
- Опишите, почему вас привлекает данная вакансия и компания.
- Расскажите о своем опыте и навыках, которые подходят под вакансию.
- Укажите, какие навыки вам не хватает и готовность их развивать.
- Подчеркните свои дополнительные преимущества, такие как смежный опыт или редкие качества.
- Пишите письмо в стиле, соответствующем отрасли и вакансии.
- Соблюдайте краткость, чтобы письмо не превышало пары абзацев.
- Завершайте письмо вежливым прощанием и возможностью для обратной связи.
- Не останавливайтесь на достигнутом и продолжайте развиваться.
'''

In [24]:
# Проходим по каждому элементу списка
for vacancy in vacancies:
    messages = [{"role": "system", "content": system_promt_2},
                {"role": "user", "content": 'ТЕКСТ РЕЗЮМЕ: ' + resume + 'ТЕКСТ ВАКАНСИИ: ' + vacancy['text']}]

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo",
                  messages=messages,
                  temperature=0
                  )
    vacancy['cover_letter_2'] = completion['choices'][0]['message']['content']

    print("name:", vacancy['name'])
    print('cover_letter_2:', vacancy['cover_letter_2'],'\n' )

name: Вакансия01.txt
cover_letter_2: Уважаемый HR-холдинг Ventra,

Я хотел бы выразить свой интерес к вакансии Data Scientist в вашей компании. Меня привлекает ваша компания и возможность применить мои навыки анализа данных и машинного обучения.

В моем резюме вы можете увидеть, что я являюсь мотивированным Junior Data Science-инженером с годовым опытом в анализе данных, разработке моделей предсказаний и машинном обучении. Я имею опыт работы с языками программирования Python и R, а также с библиотеками Pandas, NumPy и Scikit-Learn. Я также имею базовые знания SQL и понимание методов машинного обучения.

В моей текущей работе в компании XYZ Corp я занимаюсь проектированием и реализацией моделей предсказательного анализа, обработкой и анализом больших данных с использованием Python, R, SQL и Tableau. Я также строю и отлаживаю алгоритмы машинного обучения и искусственного интеллекта. Мой опыт работы в команде и хорошие коммуникативные навыки позволяют мне успешно сотрудничать с коллегами 

## Вариант 3

Текст `system_promt_3` по факту не использует никаких инструкций.

In [25]:
system_promt_3 = '''
Ты должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.
'''

In [26]:
# Проходим по каждому элементу списка
for vacancy in vacancies:
    messages = [{"role": "system", "content": system_promt_3},
                {"role": "user", "content": 'ТЕКСТ РЕЗЮМЕ: ' + resume + 'ТЕКСТ ВАКАНСИИ: ' + vacancy['text']}]

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo",
                  messages=messages,
                  temperature=0
                  )
    vacancy['cover_letter_3'] = completion['choices'][0]['message']['content']

    print("name:", vacancy['name'])
    print('cover_letter_3:', vacancy['cover_letter_3'],'\n' )

name: Вакансия01.txt
cover_letter_3: Уважаемая команда HR-холдинга Ventra,

Я хотел бы выразить свой интерес к вакансии Data Scientist, опубликованной вашей компанией. Я уверен, что мои навыки и опыт в анализе данных и машинном обучении могут быть ценными для вашей команды.

В моем предыдущем опыте работы в компании XYZ Corp я занимал должность Junior Data Scientist, где я разрабатывал и реализовывал модели предсказательного анализа. Я также обрабатывал и анализировал большие объемы данных с использованием Python, R, SQL и Tableau. Мой опыт включает построение и отладку алгоритмов машинного обучения и искусственного интеллекта. Я также имею опыт работы с библиотеками Pandas, NumPy и Scikit-learn, а также с инструментами визуализации данных, такими как Matplotlib, Seaborn и Tableau.

Я обладаю хорошими навыками анализа данных и проблемного решения, а также вниманием к деталям. Я способен работать над мультидисциплинарными проектами и быстро обучаться новым технологиям. У меня также хоро

In [27]:
vacancies

[{'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1fHhz9mwUk80msizLUH87A64knKEXWHbG',
  'name': 'Вакансия01.txt',
  'text': 'HR-холдинг Ventra\r\n\r\nО компании\r\nVentra — HR-холдинг, предлагающий инновационные решения в области аутсорсинга бизнес-процессов, HR-консалтинга и гибкой занятости. Мы обладаем широкими возможностями для решения ваших задач: полный спектр HR-сервисов, услуги аутсорсинга продаж, ИТ, процессов складской логистики и производственных функций. В июне 2022 холдинг приобрел бизнес компании Kelly в России, объединив сильнейшие на российском рынке команды в области решений по управлению персоналом и аутсорсинга. \r\n20\r\nлет компания на рынке\r\n10000\r\nсотрудников на проектах компании\r\n350\r\nклиентов из ТОП-10 индустрий\r\n5+\r\nлет каждый клиент сотрудничает с Ventra\r\n70%\r\nклиентов пользуются 2 и более услугами\r\n450 тыс.\r\nтранзакций в сутки в платформе Ventra Portal\r\n300\r\nтоп-менеджеров в закрытом клубе Ventra Lab\r\n Направления\r\nМы у

In [28]:
import csv

# Открываем CSV-файл для записи
with open('letter_options.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Записываем заголовки столбцов
    writer.writerow(["Вакансия", "Текст вакансии", "Вариант письма 1", "Вариант письма 2", "Вариант письма 3"])

    # Записываем значения из списка vacancies
    for vacancy in vacancies:
        name = vacancy.get('name', '')
        text = vacancy.get('text', '')
        cover_letter_1 = vacancy.get('cover_letter_1', '')
        cover_letter_2 = vacancy.get('cover_letter_2', '')
        cover_letter_3 = vacancy.get('cover_letter_3', '')

        writer.writerow([name, text, cover_letter_1, cover_letter_2, cover_letter_3])